In [8]:
import json
import io
from io import BytesIO
import os
import csv
from csv import reader
import requests
import pycurl
import certifi
import time
import datetime
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType

import pydub
from pydub.playback import play
from pydub import AudioSegment
import ffmpeg

from functools import partial
from itertools import repeat
import multiprocessing
from multiprocessing import Pool, freeze_support

In [2]:
workindir = r'/Users/michaelfelzan/Desktop/GEO FM'


acrcloud_accesskey = '4554490a1c83c19ea6745ed6bfbbea7d'
acrcloud_secretkey = 'HcoVKztgUltoNAwlPxpWoIJzt86HoKk9KTiByPTl'

acrcloud_config = {
    'host':'identify-eu-west-1.acrcloud.com',
    'access_key': acrcloud_accesskey,
    'access_secret': acrcloud_secretkey,
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO, # could be 'humming audio' as well
    'debug':False,
    'timeout':5 # seconds
}

ACR_recognizer = ACRCloudRecognizer(acrcloud_config)

In [3]:
def RouteToACRCloud(mp3):
    """Function that routes an mp3 file (path) to the
    ACRCloud API, in order to return the song name info.
    
    Parameters
    ------------
    mp3 : str
        Name of mp3 file (path)
        """
    buf = open(mp3,'rb').read()
    # start second will be 0; will record sampled mp3 for 10 seconds
    songread_output = ACR_recognizer.recognize_by_filebuffer(buf, 0)
    return songread_output

In [4]:
client_key = 'gQylLmyvGPfyGEvmFyRx'
client_secret = 'KJnItkPttweOtFSkqwAiTTCWJsOnIobH'

In [5]:
def RadioMP3Recorder(outpath_name, instreamurl):
    
    audio_input = ffmpeg.input(instreamurl)
    audio_output = ffmpeg.output(audio_input,
                                 outpath_name,
                                 **{'b:a': '128k'},
                                 ss=10,
                                 t=10)
    audio_output.run()

In [6]:
def RollRadio(url, name):
    timerightnow = datetime.datetime.now()
    formattedtime = timerightnow.strftime("%Y_%m_%d_T%H_%M_%S")
    
    input_stream = url
    input_mp3name = f'{name}'+f'{formattedtime}'+'.mp3'
    itersongpath = os.path.join(r'/Users/michaelfelzan/Desktop/GEO FM',
                                input_mp3name)
    print(f'~~Now recording... {input_stream}~~')
    RadioMP3Recorder(itersongpath,
                    input_stream)
    acrcloud_output = RouteToACRCloud(itersongpath)
    songread_dict = json.loads(acrcloud_output)
    
    ACR_return = 'pepe'
    
    if songread_dict['status']['msg'] == 'No result':
        ACR_return = f'No ACR return from {input_mp3name}'
        #pass
    elif songread_dict['status']['msg'] == 'May Be Mute':
        ACR_return = f'No ACR return from {input_mp3name}'
        #pass
    elif songread_dict['status']['msg'] == 'Decode Audio Error':
        ACR_return = f'No ACR return from {input_mp3name}'
        #pass
    else:
        songtitle = songread_dict['metadata']['music'][0]['title']
        songartist = songread_dict['metadata']['music'][0]['artists'][0]['name']
        songalbum = songread_dict['metadata']['music'][0]['album']['name']
        
        ACR_return = f'{input_mp3name} ~ song : {songtitle} by {songartist} from album {songalbum}'
    
    
    
        percent20song = songtitle+'%20'+songartist+'%20'+songalbum
        discogs_input = percent20song.replace(" ", "%20")
        discogs_req = ('https://api.discogs.com/database/search?q=' + 
                       discogs_input + 
                       '&key=' + 
                       client_key + 
                       '&secret=' +
                       client_secret)
        discogs_req_obj = requests.get(discogs_req)
        discogs_songinfo_dict = json.loads(discogs_req_obj.content.decode('utf-8'))
        
        if discogs_songinfo_dict['results'] == []:
            Discogs_return = f'No Discogs return from {input_mp3name}'
            #pass
        else:
            style_list = discogs_songinfo_dict['results'][0]['style']
            genre_list = discogs_songinfo_dict['results'][0]['genre']
            styleliststring = ','.join(style_list)
            genreliststring = ','.join(genre_list)
            
            #with open(station_dict[station]['txtpath'], "a+") as file_object:
                #file_object.seek(0)
                #data = file_object.read(100)
                #if len(data) > 0 :
                    #file_object.write("\n")
                #file_object.write((input_mp3name+
                                   #','+
                                   #'song='+songtitle+','+
                                   #'artist='+songartist+','+
                                   #'album='+songalbum+','+
                                   ####'genres='+genreliststring+','+
                                   #'styles='+styleliststring))
            Discogs_return = f'Discogs returned the genre(s): {styleliststring}'
                
            return( [ACR_return,Discogs_return] )

In [21]:
globals()['KEXP']

KeyError: 'KEXP'

In [22]:
KEXP

NameError: name 'KEXP' is not defined

In [20]:
array_of_stations = [('selfy',r'https://stream.wmnf.org:4443/wmnf_hd2','tbaby'),
                     ('selfy',r'https://stream.wmnf.org:4443/wmnf_hd4','tbubi'),
                     ('selfy',r'https://17553.live.streamtheworld.com/WPBBFMAACHI.aac','tibz')]
if __name__ == '__main__':
    with Pool(6) as p:
    # number of worker processes ^
        p.starmap(RollRadio, [(r'https://stream.wmnf.org:4443/wmnf_hd2','tbaby'),
                              (r'https://stream.wmnf.org:4443/wmnf_hd4','tbubi'),
                              (r'https://17553.live.streamtheworld.com/WPBBFMAACHI.aac','tibz')])

~~Now recording... https://stream.wmnf.org:4443/wmnf_hd2~~
~~Now recording... https://stream.wmnf.org:4443/wmnf_hd4~~
~~Now recording... https://17553.live.streamtheworld.com/WPBBFMAACHI.aac~~


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 470, in _handle_results
    task = get()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
TypeError: __init__() missing 2 required positional arguments: 'stdout' and 'stderr'

Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Process ForkPoolWorker-30:
Process ForkPoolWorker-28:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback 

AssertionError: Cannot have cache with result_hander not alive

Process ForkPoolWorker-27:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-6-17e5007f503c>", line 11, in RollRadio
    input_stream)
  File "<ipython-input-5-a24a112501ef>", line 9, in RadioMP3Recorder
    audio_output.run()
  File "/opt/anaconda3/lib/python3.7/site-packages/ffmpeg/_run.py", line 322, in run
    out, err = process.communicate(input)
  File "/opt/anaconda3/lib/python3.7/subprocess.py", line 931, in communicate
    self.wait()
  File "/opt/anaconda3/lib/python3.7/subprocess

In [19]:
import sys
 
 
def print_to_stderr(*a):
 
    # Here a is the array holding the objects
    # passed as the argument of the function
    print(*a, file = sys.stderr)
 
print_to_stderr("Hello World")

Hello World


In [15]:
def f(self, *args):
    arr, arg1, arg2 = args
    print(f"{arr} {arg1} {arg2}")

In [17]:
f('sdfs','fdf','sdff','sdf')

fdf sdff sdf


In [9]:
type(array_of_stations)

list

In [39]:
with open('zimbabwe sample csv.csv', newline='') as csvfile:
    zimbreader = csv.reader(csvfile, delimiter=',')
    zimb_list = list(zimbreader)
    for i in range(11):
        print(zimb_list[i][7])

http://207.180.222.146:8000/stream
http://41.174.189.194:88/broadwave.mp3?src=1&amp;rate=1
http://197.221.240.226:88/broadwave.mp3?src=1&amp;kbps=80
http://197.221.240.166:88/broadwavehigh.mp3?src=1
http://41.220.21.66:88/broadwave.mp3
http://41.220.21.70:88/broadwavehigh.mp3
https://eu4.fastcast4u.com/proxy/lancguma?mp=/1
http://41.220.21.66:88/broadwave.mp3?src=1
http://41.220.21.70:88/broadwavelow.mp3
https://edge.iono.fm/xice/159_medium.aac
https://edge.iono.fm/xice/134_medium.aac


In [ ]:
def RadioTowerInfoGetter(callsign):
    """Function that requests the HTML of
    the radio-locator.com webpage for whatever 
    radio station callsign is inputted into the function.
    
    The function parses the HTML to return a dictionary
    containing the radio stations ERP, lat/long,
    HAAT, HAGL, and HASL.
    
    Parameters
    ------------
    callsign : str
        Name of station callsign (eg. KEXP)
    """
    radiolocheaders = {
        'sec-ch-ua' : '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile' : '?0',
        'Upgrade-Insecure-Requests' : '1',
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
    }
    locatorURL = f'https://radio-locator.com/cgi-bin/finder?call={callsign}&x=0&y=0&sr=Y&s=C'
    
    radiolocreq = requests.get(locatorURL,
                               headers=radiolocheaders)
    stationHTML = radiolocreq.content.decode('utf-8')
    stationsoup = BeautifulSoup(stationHTML)
    techvalues = stationsoup.find_all("td",
                                      class_='tech_value')
    
    if techvalues == []:
        print(f"No radio tower info could be retrieved for station {callsign}")
        return False
    else:
        ERP = []
        coords = []
        heights = []
        # Parsing the HTML:
        for item in techvalues:
            for characters in item:
                for sub in characters:
                    if '" N' in sub:
                        coords.append(sub)
            if 'Watts' in characters:
                ERP.append(characters)
            elif 'meters' in characters:
                heights.append(characters)
            else:
                pass
    
        itercoords = coords[0]
        iterERP = ERP[0]
        iterHAAT = heights[0]
        iterHAGL = heights[1]
        iterHASL = heights[2]
    
        towerinfo = {
            'ERP' : iterERP,
            'Coords' : itercoords,
            'HAAT' : iterHAAT,
            'Height Above Ground Level' : iterHAGL,
            'Height Above Sea Level' : iterHASL
        }
    
        return towerinfo